In [1]:
from bpch2nc import bpch_2_netcdf
import numpy as np
import xarray as xr
import re

In [2]:
# ----------------------------------------------------------------------
# User configurable settings (EDIT THESE ACCORDINGLY)
# ----------------------------------------------------------------------

# Name of Bpch file
case_name   = 'CO2-2018-05'
directory = '/geos/u73/msadiq/GEOS-Chem/rundirs/ensemble_runs/' + case_name + '/nd51/'

first_day   = '2018-05-01'
last_day_p1 = '2019-06-01'

name_bpch1 = 'ts_satellite.S'
tinfo_file = directory + 'tracerinfo.dat'
dinfo_file = directory + 'diaginfo.dat'
#date = 20180201
#bpchfile = directory + name_bpch1+str(date)+'.bpch'

# tracerinfo.dat and diaginfo,dat fiels
#tinfo_file = directory + 'tracerinfo.dat'
#dinfo_file = directory + 'diaginfo.dat'

# Name of netCDF file
output_directory = '/geos/u73/msadiq/GEOS-Chem/rundirs/ensemble_runs/' + case_name + '/nd51/'
#ncfile = output_directory + name_bpch1 + str(date) + '.nc'

# Number of seconds in the diagnostic interval (assume 1-month)
# does not matter for CO2
interval = 86400.0 * 31.0

In [3]:
days = np.arange(first_day, last_day_p1, dtype='datetime64[D]')

for iday in np.arange(len(days)):
    day_string = days[iday] # format not right for the following function
    print('converting bpch to netcdf on day: ', day_string)
    new_day_string = re.sub("[^0-9]", "", str(day_string)) # strip off '-'s
    
    bpchfile = directory + name_bpch1 + new_day_string + '.bpch'
    ncfile = output_directory + name_bpch1 + new_day_string + '.nc'
    
    bpch_2_netcdf(bpchfile=bpchfile, 
                  tinfo_file=tinfo_file, 
                  dinfo_file=dinfo_file, 
                  ncfile=ncfile)

converting bpch to netcdf on day:  2018-01-01


KeyError: 'full_name'

In [ ]:
#ds = xr.open_dataset(ncfile)
#ds = xr.open_dataset(directory + 'GEOSChem.SpeciesConc.20180201_0000z.nc4')
#ds